# Data Processing Task using Python

I recently undertook a Data Challenge. The aim of the challenge was to practise some basic data processing and answer some questions for 3 data sets. I had 3 data sets:


    1. leads.csv. This is a list of fictitious company directors, with some basic data about them and their company.
    2. calls.csv. This is a list of fictitious calls made by an outbound call centre. The call centre consists of several agents, who make calls one after the other. They don’t get to choose who to call, the system does. The objective of the call is to get the lead to signup on the website. When they finish a call, they mark down the outcome, from a fixed list of possible outcomes. Note that a single lead may be called multiple times.
    3. signups.csv. This is a list of leads who signed up after being called by someone from the call centre. Each signup was risk assessed and either approved or rejected for a loan.

I then had some questions I wanted to answer: 

1.	Which agent made the most calls?
2.	For the leads that received one or more calls, how many calls were received on average?
3.	For the leads that signed up, how many calls were received, on average?
4.	Which agent had the most signups? Which assumptions did you make? (note that there is a many-to-one relationship between calls and leads)
5.	Which agent had the most signups per call?
6.	A lead from which region is most likely to be “interested” in the product?
7.	A lead from which sector is most likely to be “interested” in the product?
8.	Given a lead has already expressed interest and signed up, 
a.	signups from which region are most likely to be approved?
b.	Is this statistically significant? Why?
9.	Suppose you wanted to pick the 1000 leads most likely to sign up (who have not been called so far), based only on age, sector and region.
a.	What criteria would you use to pick those leads?
b.	In what sense are those an optimal criteria set?
c.	How many signups would you expect to get based on those called leads, assuming they were being called by random agents?
d.	If you could choose the agents to make those calls, who would you choose? Why?


For this task, I choose to use Python along with pandas, numpy, scipy and statstools.

To start off with, I need to read in the csv files, this is easily done with pandas.

In [1]:
import pandas as pd

calls_df = pd.read_csv('calls.csv')
leads_df = pd.read_csv('leads.csv')
signups_df = pd.read_csv('signups.csv')

**Question 1: Which agent made the most calls?**
 
This is easily answered by taking the calls data and looking at the modal value of the agent column

In [2]:
mostCalls = calls_df.mode()
print(mostCalls.iloc[0])

Phone Number        4.66939e+11
Call Outcome    CALL BACK LATER
Agent                    orange
Call Number                   0
Name: 0, dtype: object


This gives the answer as Orange

**Question 2: For the leads that received one or more calls, how many calls were received on average?**

For this question we want to look at our calls data again. 

We want to exclude the calls that were classed as a 'dead line' as noone received a call here.

In [3]:
receivedCalls = calls_df[calls_df['Call Outcome'] != 'DEAD LINE']

We can then calculate the average amount of calls for the remaining phone numbers

In [12]:
signupCalls = receivedCalls['Phone Number'].value_counts()
print("Average number of calls: " + str(signupCalls.mean()))

Average number of calls: 1.893150684931507


**Question 3 - For the leads that signed up, how many calls were received, on average?**

For this question we need to combine our datasets. Before we can do this we need to get the column names to match up - the same data is called Name in one dataset and Lead in another.

In [9]:
signups_df.columns = ['Name', 'Approval Decision']

We can then merge the data:

In [10]:
whoCalled = pd.merge(leads_df, signups_df)
callsList = pd.merge(whoCalled, calls_df)

And take the mean occurance of each phone number:

In [13]:
callsTotals = callsList['Phone Number'].value_counts()
print("Average number of calls: " + str(callsTotals.mean()))

Average number of calls: 2.0989583333333335


**Question 4 - Which agent had the most signups?**

For this question we make the assumption that the leads signed up after an ‘interested’ call and the agent who’s call resulted in ‘interested’ deserved the credit for the sign up. 

We can then look at the occurances of interested and take the modal:

In [14]:
interestedLeads = callsList[callsList['Call Outcome'] == 'INTERESTED']
agentCount = interestedLeads['Agent'].value_counts()
print(agentCount)

red       316
orange    284
green      67
blue       52
black      49
Name: Agent, dtype: int64


The answer is red.

**Question 5 - Which agent had the most signups per call? **

For this question we need to take the total number of 'interested' calls for each agent (from question 4) and then divide that by the total number of calls they took to get the ratio of signups ber call.

In [15]:
totalAgentCalls = calls_df['Agent'].value_counts()
agentSignups = pd.DataFrame({'Agent': agentCount.index, 'Sign Ups': agentCount.values})
agentCalls = pd.DataFrame({'Agent': totalAgentCalls.index, 'Call Numbers': totalAgentCalls.values})
total = pd.merge(agentSignups, agentCalls)
total['z'] = total['Sign Ups']/total['Call Numbers']
print(total)

    Agent  Sign Ups  Call Numbers         z
0     red       316          1478  0.213802
1  orange       284          2234  0.127126
2   green        67           339  0.197640
3    blue        52           199  0.261307
4   black        49           750  0.065333


z gives the signups ratio and from this we can see the agent with the highest signups per call is blue.

**Question 6 - A lead from which region is most likely to be “interested” in the product?**

For this question we want to look at leads and calls:

In [16]:
callsLeadMerge = pd.merge(leads_df, calls_df)

We want to take all the 'interested' calls and then take count of the regions:

In [17]:
interested = callsLeadMerge[callsLeadMerge['Call Outcome'] == 'INTERESTED']
mostInterestedRegion = interested['Region'].value_counts()
print(mostInterestedRegion)

north-west          365
south-west          161
midlands            150
north-east          139
scotland            137
south-east          136
south                62
london               56
wales                50
northern-ireland     40
Name: Region, dtype: int64


Leads from the North-West are most likely to be 'interested' in the product.

**Question 7 - A lead from which sector is most likely to be “interested” in the product?**

This question follows the same precedure as Question 6:

In [19]:
mostInterestedSector = interested['Sector'].value_counts()
print(mostInterestedSector)

consultancy      301
retail           290
food             261
wholesale        233
entertainment    135
construction      46
agriculture       30
Name: Sector, dtype: int64


Giving consultancy as the sector most likely to be interested in the product.

**Question 8 a. signups from which region are most likely to be approved?**

For this question we want to take a look at all our datasets:

In [21]:
callsSignsMerge = pd.merge(callsLeadMerge, signups_df)

and then select the data as required:

In [24]:
approved = callsSignsMerge[callsSignsMerge['Approval Decision'] == 'APPROVED']
approvedInterested = approved[approved['Call Outcome'] == 'INTERESTED']
mostApprovedRegion = approvedInterested['Region'].value_counts()
print(mostApprovedRegion)

north-west          95
scotland            37
south-east          29
midlands            26
south-west          25
north-east          20
south               12
northern-ireland     6
wales                5
london               2
Name: Region, dtype: int64


Giving the North-West as the most approved region.

**Question 8 b. Is this statistically significant?**

This question is more complicated. A great way to model ppredict proabilites of a value being in one of two categories (in this case approved or not approved) is to use a Probit model. 